In [1]:
from pydeseq2.ds import DeseqStats
from utils.utils import load_model, save_df

In [2]:
# load model
dds = load_model(['species'])

#  DeseqStats to perform the statistical analysis
ds = DeseqStats(dds, contrast=['species', 'hypochondriacus','caudatus'], alpha=0.01, independent_filter=False)   # make padj deterministic
ds.summary()

Successfully loaded model: dds_species


Running Wald tests...


Log2 fold change & Wald test p-value: species hypochondriacus vs caudatus
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
AHq000001   423.562228       -0.071131  0.203305 -0.349873  0.726434  0.818046
AHq000002  1575.517497        1.170735  0.417829  2.801947  0.005080  0.022686
AHq000003   352.963514        0.162567  0.245005  0.663523  0.506996  0.645568
AHq000004   340.614957        0.150058  0.350274  0.428403  0.668358  0.774188
AHq000005   668.719091        0.425988  0.346249  1.230295  0.218587  0.358085
...                ...             ...       ...       ...       ...       ...
AHq024993    17.464828        0.591391  1.754689  0.337035       NaN       NaN
AHq024994    18.575629        2.115156  1.706476  1.239488       NaN       NaN
AHq025019   192.046205        0.356398  0.350318  1.017355  0.308985  0.457162
AHq025020  1045.351245        1.044924  0.242998  4.300135  0.000017  0.000248
AHq025042    13.565600        2.027804  1.971139  1.02874

... done in 5.09 seconds.



In [3]:
# Extract the results as a pandas DataFrame
results_df = ds.results_df
save_df(results_df, 'statistical_results_hypochondriacus_caudatus_df.csv')

In [4]:
def significant_genes_count(results_df,
                             padj_treshold: float = 0.05,
                             log2FoldChange_treshold: float = 1) -> int:
    """
    Count genes that meet standard DE-analysis significance criteria.

    A gene is deemed significant when:
      * its adjusted p-value (`padj`) is below ``padj_treshold``, **and**
      * the absolute log2 fold-change (`log2FoldChange`) exceeds
        ``log2FoldChange_treshold``.

    Parameters
    ----------
    results_df : pandas.DataFrame
        Data frame (e.g. `DeseqStats.results_df`) containing at least the
        columns ``'padj'`` and ``'log2FoldChange'``.
    padj_treshold : float, optional
        Adjusted p-value cutoff; default is ``0.05``.
    log2FoldChange_treshold : float, optional
        Minimum absolute log2 fold-change; default is ``1``.

    Returns
    -------
    int
        Number of genes satisfying both criteria.

    Examples
    --------
    >>> n_sig = significant_genes_count(res_df, 0.01, 1.5)
    >>> print(f"{n_sig} genes are significant.")
    """
    return len(
        results_df[
            (results_df["padj"] < padj_treshold)
            & (abs(results_df["log2FoldChange"]) > log2FoldChange_treshold)
        ]
    )

In [5]:
sig_genes_before_shrink = significant_genes_count(results_df)
print("Significant genes before shrinkage:", sig_genes_before_shrink)

Significant genes before shrinkage: 4234


In [6]:
# apply lfc shrinkage - changes log2FoldChange - padj remain the same
ds.lfc_shrink(coeff=dds.varm['LFC'].keys()[-1])

Fitting MAP LFCs...


Shrunk log2 fold change & Wald test p-value: species hypochondriacus vs caudatus
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
AHq000001   423.562228       -0.065732  0.200792 -0.349873  0.726434  0.818046
AHq000002  1575.517497        1.019457  0.420653  2.801947  0.005080  0.022686
AHq000003   352.963514        0.150430  0.242027  0.663523  0.506996  0.645568
AHq000004   340.614957        0.128459  0.339802  0.428403  0.668358  0.774188
AHq000005   668.719091        0.374397  0.339021  1.230295  0.218587  0.358085
...                ...             ...       ...       ...       ...       ...
AHq024993    17.464828        0.342442  1.653925  0.337035       NaN       NaN
AHq024994    18.575629        0.357319  1.607735  1.239488       NaN       NaN
AHq025019   192.046205        0.306580  0.342169  1.017355  0.308985  0.457162
AHq025020  1045.351245        0.998641  0.243465  4.300135  0.000017  0.000248
AHq025042    13.565600        0.258668  1.661159  

... done in 8.00 seconds.



In [7]:
sig_genes_after_shrink = significant_genes_count(results_df)
print("Significant genes after shrinkage:", sig_genes_after_shrink)
print("Difference to number of genes before lfc shrinkage:", sig_genes_before_shrink - sig_genes_after_shrink)

Significant genes after shrinkage: 3854
Difference to number of genes before lfc shrinkage: 380


In [8]:
# Manually filtering results for significant DEG based on adjusted p-value after LFC shrinkage
significant_results = results_df[(results_df['padj'] < 0.05) & (abs(results_df['log2FoldChange']) > 1)]
print(significant_results)

              baseMean  log2FoldChange     lfcSE       stat        pvalue  \
AHq000002  1575.517497        1.019457  0.420653   2.801947  5.079525e-03   
AHq000022   217.034515       -1.128378  0.305612  -3.933239  8.380871e-05   
AHq000038   253.044158       -2.211554  0.610289  -4.046933  5.189305e-05   
AHq000052   185.638237       -4.880228  0.860050  -5.937624  2.891814e-09   
AHq000057   199.628097        1.491808  0.281231   5.584511  2.343585e-08   
...                ...             ...       ...        ...           ...   
AHq024861   107.696769        5.117326  0.491480  10.802272  3.357884e-27   
AHq024869   177.215456        3.567805  0.821545   4.933737  8.067112e-07   
AHq024875  6773.255081        1.995769  0.952468   2.951113  3.166306e-03   
AHq024922  1167.485887       13.521477  1.316624  10.852318  1.944300e-27   
AHq024948    55.005518        1.080197  0.315839   3.705905  2.106372e-04   

                   padj  
AHq000002  2.268613e-02  
AHq000022  9.220857e-04

In [9]:
save_df(significant_results, 'significant_DEG_results_hypochondriacus_caudatus_df.csv')

## Number of positive log2FoldChange (>0) and number of log2FoldChange (<0)

In [10]:
f'{len(significant_results[significant_results['log2FoldChange'] > 0])} genes have a positive log2FoldChange.'

'1625 genes have a positive log2FoldChange.'

In [11]:
f'{len(significant_results[significant_results['log2FoldChange'] < 0])} genes have a negative log2FoldChange.'

'2229 genes have a negative log2FoldChange.'